In [10]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.metrics import mean_squared_error
plt.rcParams['font.family'] = 'Apple SD Gothic Neo'
plt.rcParams['axes.unicode_minus'] = False
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정
# pd.read_csv() 함수를 사용해서 데이터를 읽어오는 코드입니다.
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# pd.read_csv() 함수를 사용해서 데이터를 읽어오는 코드입니다.
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [11]:
import numpy as np
from sklearn.metrics import (
    log_loss,
    accuracy_score,
    classification_report,
    mean_squared_error,
    r2_score,
)

def evaluate_model(model, X_valid, y_valid, name="model", verbose=True):
    y_pred = model.predict(X_valid)

    # Log Loss (로스 로스, log_loss)
    loss = None
    if hasattr(model, "predict_proba"):
        try:
            y_proba = model.predict_proba(X_test)
            loss = log_loss(X_valid, y_proba)
        except Exception:
            loss = None

    # R² (결정계수, R-squared)
    r2 = r2_score(y_valid, y_pred)

    # 정확도(Accuracy)
    acc = None
    cls_report = None
    try:
        acc = accuracy_score(y_valid, y_pred)
        cls_report = classification_report(y_valid, y_pred)
    except Exception:
        # 회귀모델(continuous target)일 때는 여기로 옴
        pass

    # MSE / RMSE
    mse = mean_squared_error(y_valid, y_pred)
    rmse = np.sqrt(mse)

    # ------------ 출력 (verbose=True일 때만) ------------
    if verbose:
        print("=" * 40)
        print(f"[{name}] 모델 성능 평가")
        print("=" * 40)

        if loss is not None:
            print("Log Loss:", loss)

        print(f"{name:15s} R score(): {r2:.4f}")

        if acc is not None:
            print("정확도:", acc)
            if cls_report is not None:
                print(cls_report)

        print("-" * 40)
        print(f"MSE (평균 제곱 오차): {mse:.3f}")
        print(f"RMSE (평균 제곱근 오차): {rmse:.3f}")
        print(f"R² Score (결정계수): {r2:.3f}")
        print("=" * 40)

    # 핵심: 정확도(없으면 None)를 리턴
    return acc

In [12]:
train.rename(columns={'혈압': '맥압'}, inplace=True)
test.rename(columns={'혈압': '맥압'}, inplace=True)

In [13]:
train.loc[train['시력'] > 2, '시력'] = 2.0
train.loc[train['혈청 크레아티닌'] > 1.5, '혈청 크레아티닌'] = 1.5
train.loc[train['요 단백'] > 4, '요 단백'] = 4.0
train.loc[train['저밀도지단백'] > 150, '저밀도지단백'] = 150
train.loc[train['고밀도지단백'] > 100, '고밀도지단백'] = 100
train.loc[train['중성 지방'] > 200, '중성 지방'] = 200
train.loc[train['공복 혈당'] > 200, '공복 혈당'] = 200

train.loc[train['저밀도지단백'] < 22, '저밀도지단백'] = 22
train.loc[train['고밀도지단백'] < 22, '고밀도지단백'] = 22


In [15]:
test.loc[test['시력'] > 2, '시력'] = 2.0
test.loc[test['혈청 크레아티닌'] > 1.5, '혈청 크레아티닌'] = 1.5
test.loc[test['요 단백'] > 4, '요 단백'] = 4.0
test.loc[test['저밀도지단백'] > 150, '저밀도지단백'] = 150
test.loc[test['고밀도지단백'] > 100, '고밀도지단백'] = 100
test.loc[test['중성 지방'] > 200, '중성 지방'] = 200
test.loc[test['공복 혈당'] > 200, '공복 혈당'] = 200
test.loc[test['간 효소율'] > 10, '간 효소율'] = 10

test.loc[test['저밀도지단백'] < 22, '저밀도지단백'] = 22
test.loc[test['고밀도지단백'] < 22, '고밀도지단백'] = 22


In [17]:
train['저밀도지단백_고밀도지단백_낮음'] = train['저밀도지단백'] / train['고밀도지단백']
train['고밀도지단백_저밀도지단백_높음'] = train['고밀도지단백'] / train['저밀도지단백']
train['중성지방_고밀도지단백_2이하'] = train['중성 지방'] / train['고밀도지단백']
train['true_BMI'] = train['몸무게(kg)'] / ((train['키(cm)'] / 100) ** 2)
train['총 콜레스테롤'] = train['저밀도지단백'] +train['고밀도지단백'] + (train['중성 지방'] / 5)



In [18]:
test['저밀도지단백_고밀도지단백_낮음'] = test['저밀도지단백'] / test['고밀도지단백']
test['고밀도지단백_저밀도지단백_높음'] = test['고밀도지단백'] / test['저밀도지단백']
test['중성지방_고밀도지단백_2이하'] = test['중성 지방'] / test['고밀도지단백']
test['true_BMI'] = test['몸무게(kg)'] / ((test['키(cm)'] / 100) ** 2)
test['총 콜레스테롤'] = test['저밀도지단백'] +test['고밀도지단백'] + (test['중성 지방'] / 5)


In [19]:
train.drop(columns=['BMI','콜레스테롤'], inplace=True)

In [20]:
test.drop(columns=['BMI','콜레스테롤'], inplace=True)

In [21]:
X = train.drop(['label','ID',], axis = 1)
y = train['label']
X_test = test.drop(['ID',], axis = 1)

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


---

# 모델링

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

probs = models.predict_proba(X_valid)[:, 1]

best_acc = 0
best_thr = 0.5

for thr in np.linspace(0.1, 0.9, 17):  # 0.1 ~ 0.9 사이 0.05 간격
    y_pred_thr = (probs >= thr).astype(int)
    acc = accuracy_score(y_valid, y_pred_thr)
    if acc > best_acc:
        best_acc = acc
        best_thr = thr

print("Best threshold:", best_thr)
print("Best val accuracy:", best_acc)